<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240320%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240320T125036Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D748fa3189c96f42b25895a5bb945182d2a31e07bf1d49090783a816681be0fedf0912960ac338c1678bf74a5b9e9f82615a0dcc14f69d644cfbf90b20a9bfc763f97d52208cebaa2adb37f2b7ae914707ae7495383c75063346a2d66a8e93f56b77b35ed0b8c577cbc7e106d9c3cb2bde3d829e46cf23c7b86e81a1542a5ea61ee9babc7ccd85c965f5a1432538b8c0546e8bc6ce9cae777262d06632e7eb98dc35b90efd4bc5d582532666c26dec661160d0d8e5f4c1c4cc749bbe072e672647a53903ec1d19a9c6f4ff1d5b77ec136642a0c32cc503a5f1230a2c867ad773c0d4c440719d0475302a5c306a15b3c80a815d7ef42723de416d4bf7387d5a497,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240320%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240320T125037Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6c067764b38bd978f1d22556c42aba91f2c7eda2061c612e17cd1e7f480508c6a52bcd401561e33b473e3eca4bc5a01fdb39f54dc8c68efd9751ac28e0f17a9c42e89f82f4c42a11a567e3a1606df8bdf84b2214efc42b0582dd5a157459e53c75ffa8b3065c9b2a0f128c3a28ff4f98a0d837dc2ef37e9617134cb33ccebca181a07b149f3a72f97f3c5ad03eb30cf34d9ade81c4a8caaa77435675619e74ae80c6af87d90b481117561c5232bb83a31548a3eb14113a35241594a6874cc00ab36e133ac51437debab2f5b92dda079625f8e48242af11f7b5da596e106fe6c1e13a8e8dddacc6af4cb3a0b57781e6c8e5eb5261b34fb66274bc71e9e39f4188'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/2b/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b/2/gemma-2b.gguf
/kaggle/input/gemma/transformers/2b/2/config.json
/kaggle/input/gemma/transformers/2b/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/tokenizer.json
/kaggle/input/gemma/transformers/2b/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b/2/.gitattributes
/kaggle/input/gemma/transformers/2b/2/tokenizer.model
/kaggle/input/gemma/transformers/2b/2/generation_config.json
/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt


# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powereg by [Gemma 7B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the latest

In [ ]:
!pip install --upgrade langchain langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-cloud-bigquery to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_google_vertexai import GemmaChatLocalHF, GemmaLocalHF
from kaggle_secrets import UserSecretsClient
access_token_read = UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")

In [ ]:
hf_access_token = access_token_read
model_name: str = "google/gemma-2b"  # @param {type:"string"}

In [ ]:
llm = GemmaLocalHF(model_name="google/gemma-2b", hf_access_token=hf_access_token)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
output = llm.invoke("what is the time cut of training dataset of Gemma google models?", max_tokens=200)
print(output)

what is the time cut of training dataset of Gemma google models?

Hi @sagar.sahu,

The training time of the models is dependent on the model and the dataset.

For example, the training time for the <code>bert-base-uncased</code> model is 10 minutes.

The training time for the <code>bert-base-cased</code> model is 10 minutes.

The training time for the <code>bert-base-multilingual-uncased</code> model is 10 minutes.

The training time for the <code>bert-base-multilingual-cased</code> model is 10 minutes.

The training time for the <code>bert-large-uncased</code> model is 10 minutes.

The training time for the <code>bert-large-cased</code> model is 10 minutes.

The training time for the <code>bert-large-multilingual-uncased</code> model is


In [ ]:
output = llm.invoke("What is the data cut-off date for the Gemma Google models training dataset?", max_tokens=200)
print(output)

What is the data cut-off date for the Gemma Google models training dataset?

The data cut-off date for the Gemma Google models training dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models validation dataset?

The data cut-off date for the Gemma Google models validation dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models test dataset?

The data cut-off date for the Gemma Google models test dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models development dataset?

The data cut-off date for the Gemma Google models development dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models training dataset


In [ ]:
output = llm.invoke("What is the most pythn command used?", max_tokens=200)
print(output)

What is the most pythn command used?

Answer:

Step 1/2
First, we need to understand what a Python command is. In Python, a command is a piece of code that is executed by the interpreter to perform a specific task. Some common Python commands include: - import: This command is used to import modules or packages into the current Python session. - print: This command is used to display text or data on the screen. - input: This command is used to prompt the user for input. - range: This command is used to create a sequence of numbers. - list: This command is used to create a list of items. - dict: This command is used to create a dictionary of key-value pairs. - for: This command is used to iterate over a sequence of items. - while: This command is used to execute a block of code repeatedly until a condition is met. - if: This command is used to execute a block of


## Requirements

## Data

## RAG

## Chatbot

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0

In [ ]:
# Setup the environment
!pip install --upgrade huggingface_hub
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
access_token_read = UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")
login(token = access_token_read)
#!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
#!pip install --upgrade transformers
!pip install transformers==4.33.1
from transformers import AutoTokenizer, AutoModelForCausalLM

import torch
from torch.utils.data import Dataset
from typing import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 6.4 MB/s eta 0:00:0000:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 12.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2

In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1390, in _get_module
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'transformers.models.gemma.configuration_gemma'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34/682258503.py", line 3, in <module>
    model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=t

In [ ]:
import pkg_resources
installed_packages = pkg_resources.working_set
installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in installed_packages])
for m in installed_packages_list:
    print(m)


/tmp/ipykernel_34/2312216702.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


absl-py==1.4.0
accelerate==0.27.2
access==1.1.9
affine==2.4.0
aiobotocore==2.11.2
aiofiles==22.1.0
aiohttp-cors==0.7.0
aiohttp==3.9.1
aioitertools==0.11.0
aiorwlock==1.3.0
aiosignal==1.3.1
aiosqlite==0.19.0
albumentations==1.4.0
alembic==1.13.1
altair==5.2.0
annotated-types==0.6.0
annoy==1.17.3
anyio==4.2.0
apache-beam==2.46.0
aplus==0.11.0
appdirs==1.4.4
archspec==0.2.2
argon2-cffi-bindings==21.2.0
argon2-cffi==23.1.0
array-record==0.5.0
arrow==1.3.0
arviz==0.17.0
astroid==3.0.3
astropy-iers-data==0.2024.2.19.0.28.47
astropy==6.0.0
asttokens==2.4.1
astunparse==1.6.3
async-lru==2.0.4
async-timeout==4.0.3
attrs==23.2.0
audioread==3.0.1
autopep8==2.0.4
babel==2.14.0
backoff==2.2.1
bayesian-optimization==1.4.3
bayespy==0.5.28
beatrix-jupyterlab==2023.128.151533
beautifulsoup4==4.12.2
bidict==0.23.1
biopython==1.83
bitsandbytes==0.43.0
blake3==0.2.1
bleach==6.1.0
blessed==1.20.0
blinker==1.7.0
blis==0.7.10
blosc2==2.5.1
bokeh==3.3.4
boltons==23.1.1
boruta==0.3
boto3==1.26.100
botocore==1.3

In [ ]:

# Load the model
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/2b/2")
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/2b/2")
# Use the model
input_text = "What is the best thing about Kaggle?"
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1390, in _get_module
    if "torch" in backends and "tf" not in backends and not is_torch_available() and is_tf_available():
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py", line 37, in <module>
    from ..utils import ModelOutput, is_accelerate_available, i